In [21]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("lr_example").getOrCreate()

from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [3]:
df_data = spark.read.csv('../Arquivos/Ecommerce_Customers.csv', inferSchema=True, header=True)

In [4]:
df_data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [19]:
df_data.toPandas().head(2)

,Email,Address,Avatar,Avg Session Length,Time on App,Time on Website,Length of Membership,Yearly Amount Spent
0,mstephenson@fernandez.com,"835 Frank TunnelWrightmouth, MI 82180-9605",Violet,34.497268,12.655651,39.577668,4.082621,587.951054
1,hduke@hotmail.com,"4547 Archer CommonDiazchester, CA 06566-8576",DarkGreen,31.926272,11.109461,37.268959,2.664034,392.204933


In [11]:
len(df_data.columns)

8

In [28]:
df_data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [27]:
assembler = VectorAssembler(inputCols=df_data.columns[3:7], outputCol='features')

In [29]:
output = assembler.transform(df_data)

In [30]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [31]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [32]:
final_data = output.select('features', 'Yearly Amount Spent')

In [33]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [34]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [35]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                345|
|   mean|  496.5439486712685|
| stddev|  83.42288334671561|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [36]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                155|
|   mean|  505.4797215335974|
| stddev|  69.15824390410444|
|    min|  304.1355915788555|
|    max|  669.9871405017029|
+-------+-------------------+



In [37]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [38]:
lr_model = lr.fit(train_data)

In [39]:
test_results = lr_model.evaluate(test_data)

In [41]:
test_results.residuals.show();

+--------------------+
|           residuals|
+--------------------+
| 0.13656145552789667|
|  10.741860554070058|
|  -6.328910231934117|
|  -7.005558916151358|
|  10.975842594275491|
|-0.16597677907532216|
|  0.7962932593991354|
|  -5.699962022369277|
| -14.346527535052473|
|   18.14144080538881|
| -3.8361927277799737|
|  -6.789504522193852|
| -4.1897124961083705|
| -11.959041634036339|
|   -4.58708164457903|
| -17.102301042799184|
| -2.9819360097162075|
|   8.024518598475481|
| -1.3945236237485688|
|  -8.971827209672426|
+--------------------+
only showing top 20 rows



In [42]:
test_results.rootMeanSquaredError

9.872554976929095

In [43]:
test_results.r2

0.9794892195857408

In [44]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [45]:
unlabeled_data = test_data.select('features')

In [46]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.5743636841713...|
|[30.7377203726281...|
|[31.0613251567161...|
|[31.1280900496166...|
|[31.1695067987115...|
|[31.2606468698795...|
|[31.3895854806643...|
|[31.5257524169682...|
|[31.5741380228732...|
|[31.6005122003032...|
|[31.6253601348306...|
|[31.7207699002873...|
|[31.7656188210424...|
|[31.8093003166791...|
|[31.8124825597242...|
|[31.8164283341993...|
|[31.8186165667690...|
|[31.8512531286083...|
|[31.8627411090001...|
|[31.8854062999117...|
+--------------------+
only showing top 20 rows



In [48]:
predictions = lr_model.transform(unlabeled_data)

In [49]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.5743636841713...|441.92785230253776|
|[30.7377203726281...| 451.0388816421598|
|[31.0613251567161...| 493.8843682898357|
|[31.1280900496166...|  564.258245663206|
|[31.1695067987115...| 416.3806882080173|
|[31.2606468698795...| 421.4926080360267|
|[31.3895854806643...|409.27331780058375|
|[31.5257524169682...|449.66558883225116|
|[31.5741380228732...| 558.7557996956393|
|[31.6005122003032...| 461.0314106857081|
|[31.6253601348306...|380.17309348470417|
|[31.7207699002873...| 545.5644380002168|
|[31.7656188210424...| 500.7437941317155|
|[31.8093003166791...| 548.7309409968775|
|[31.8124825597242...|397.39742662837625|
|[31.8164283341993...| 518.2247925464555|
|[31.8186165667690...|449.40060937985186|
|[31.8512531286083...| 464.9677280683229|
|[31.8627411090001...| 557.6926647977953|
|[31.8854062999117...|399.07510018214793|
+--------------------+------------